In [1]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import PIL.Image as Image
import torch.nn as nn
import torch.optim as optim
import sys

In [2]:
sys.path.insert(0, '../src')
from bird_dataset import *

In [3]:
bd = BirdDataset()

In [4]:
fname = 'classes-subset'
with open(f'../CUB_200_2011/{fname}.txt') as f:
    class_dict = {int(line.split(' ')[0]):i for i, line in enumerate(f.readlines())}

In [5]:
list_classes = list(class_dict.keys())
filt_images = []
for key in list(bd.images.keys()):
    if bd.images[key]['class_label'] in list_classes:
        bd.images[key]['image_id'] = key
        filt_images.append(bd.images[key])

In [17]:
# len(filt_images)

In [18]:
# bd.images

In [4]:
bd.train_indices

array([8760, 5015, 4372, ..., 2085, 1285, 7390])

In [5]:
# with open(f'../CUB_200_2011/classes-subset.txt') as f:
#     for line in f.readlines():
#         print(line.split(' '))
# #     try:class_dict = {int(line.split(' ')[0]):i for i, line in enumerate(f.readlines())}
# #     except: 
# #         print(int(line.split(' ')))
# #         raise ValueErro

In [28]:
# filt_images

In [49]:
class XAI_Birds_Dataset(Dataset):
    def __init__(self, bd:BirdDataset, images:list, subset=True, transform=None, train=True, val=False, trainset=None):
        self.bd = bd
        self.transform = transform
        self.subset = subset
        self.train = train
        self.val = val
        np.random.seed(42)
#         if self.train: self.train_test_indices = self.bd.train_indices
#         else: self.train_test_indices = self.bd.test_indices
        
        if self.subset: self.class_dict = self._set_classes('classes-subset')
        else: self.class_dict = self._set_classes('classes')
#         self.images = self.load_images()
        self.images = images
        if self.train: 
            train_indices = np.random.choice(range(len(self.images)), int(len(self.images)*0.8), replace=False)
            print(train_indices)
            self.images = np.random.choice(self.images, int(len(self.images)*0.8), replace=False)
            self.ids = [i['image_id'] for i in self.images]
            
        if self.val:
            train_indices = np.random.choice(range(len(self.images)), int(len(self.images)*0.8), replace=False)
            print(train_indices)
            img_pd = pd.Series(dict(zip(range(len(self.images)), self.images)))
            self.images = img_pd.loc[list(set(range(len(self.images))) - set(train_indices))].tolist()
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
#         img_id = list(self.images.keys())[idx]
        img_path = os.path.join(self.bd.img_dir, self.images[idx]['filepath'])
        image = Image.open(img_path)
        label = self.class_dict[self.images[idx]['class_label']]
        sample = {'image': image, 'label':label}
        
        if self.transform:
            sample['image'] = self.transform(sample['image'])
        return sample
    def _set_classes(self, fname):
        with open(f'../CUB_200_2011/{fname}.txt') as f:
            class_dict = {int(line.split(' ')[0]):i for i, line in enumerate(f.readlines())}
        return class_dict
    
#     def load_images(self):
#         images = {}
#         for key in self.bd.images:
#             class_label = self.bd.images[key]['class_label']
#             if class_label in list(self.class_dict.keys()) and class_label in self.train_test_indices:
#                 images[key] = self.bd.images[key] 
#         return images

In [138]:
# If there are GPUs, choose the first one for computing. Otherwise use CPU.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# If 'cuda:0' is printed, it means GPU is available.

cuda:0


In [169]:
transf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
train_bird_dataset = XAI_Birds_Dataset(bd, transform=transf, train=True, images=filt_images)
val_bird_dataset = XAI_Birds_Dataset(bd, transform=transf, train=False, val=True, images=filt_images)

[ 985 1502  711 ...  546  658 1494]
[ 985 1502  711 ...  546  658 1494]


In [170]:
# for i in val_bird_dataset:
#     print(i)

In [171]:
len(train_bird_dataset)

1466

In [172]:
len(train_bird_dataset) + len(val_bird_dataset)

1833

In [173]:
len(val_bird_dataset)

367

In [174]:
vgg16 = models.vgg16_bn(pretrained=True)
num_feats = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1]
features.extend([nn.Linear(num_feats, len(train_bird_dataset.class_dict))])
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier

In [175]:
# num_feats = vgg16.classifier[6].in_features
# features = list(vgg16.classifier.children())[:-1]
# features.extend([nn.Linear(num_feats, len(train_bird_dataset.class_dict))])
# vgg16.classifier = nn.Sequential(*features) # Replace the model classifier

In [176]:

if torch.cuda.is_available():
    vgg16.cuda()

In [177]:
# indices = list(range(len(bird_dataset)))
# train_idx = np.random.choice(indices, size=int(len(bird_dataset)*.8), replace=False)
# test_idx = list(set(indices) - set(train_idx))

In [178]:
# DataLoader?

In [179]:
batch_size = 4
trainloader = DataLoader(train_bird_dataset, batch_size=batch_size, shuffle=True)
valloader = DataLoader(val_bird_dataset, batch_size=batch_size, shuffle=True)

In [180]:
# for i, data in enumerate(trainloader, 0):
#     print(data)

In [181]:
loss_func = nn.CrossEntropyLoss()
opt = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

In [182]:
len(train_bird_dataset) / 60

24.433333333333334

In [ ]:
%%time
avg_losses = []
avg_val_losses = []
epochs = 500
print_freq = 100
val_acc = []
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs.
        inputs, labels = data['image'], data['label']

        # Move the inputs to the specified device.
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients.
        opt.zero_grad()

        # Forward step.
        outputs = vgg16(inputs)
        loss = loss_func(outputs, labels)
        
        # Backward step.
        loss.backward()
        
        # Optimization step (update the parameters).
        opt.step()

        # Print statistics.
        running_loss += loss.item()
#         print(outputs)
        if i % print_freq == print_freq - 1: # Print every several mini-batches.
            avg_loss = running_loss / print_freq
            print('[epoch: {}, i: {:5d}] avg mini-batch loss: {:.3f}'.format(
                epoch, i, avg_loss))
            avg_losses.append(avg_loss)
            running_loss = 0.0
            
        
    vgg16.eval()
    with torch.no_grad():
        num_correct=0
        val_losses = []
        data_iter = iter(valloader)
        for val_data in data_iter:
            val_inputs, val_labels = val_data['image'].cuda(), val_data['label'].cuda()
#                     print('val inputs: ',val_inputs.size())
#                     print('val labels: ',val_labels)
#                     print('inputs: ',inputs.size())
            val_outputs = vgg16(val_inputs)
#                     print('val outputs: ',val_outputs)
#                     print('val loss: ',nn.CrossEntropyLoss()(val_outputs, val_labels))
            opt.zero_grad() #zero the parameter gradients
            _, predicted = torch.max(outputs, 1)
            #     print(predicted)
            # print(labels)
            # print(predicted)
#             num_correct_k += topk_accuracy(k, labels, outputs)
            num_correct += sum(np.array(labels.cpu())==np.array(predicted.cpu()))
            val_losses.append(loss_func(val_outputs, val_labels).item())
        acc = num_correct/(len(data_iter)*batch_size)
        val_acc.append(acc)
        print('Validation accuracy:',acc)
        print('Average validation loss:',np.mean(val_losses))
        avg_val_losses.append(np.mean(val_losses))
    vgg16.train()
print('Finished Training.')

[epoch: 0, i:    99] avg mini-batch loss: 3.653
[epoch: 0, i:   199] avg mini-batch loss: 3.488
[epoch: 0, i:   299] avg mini-batch loss: 3.465
Validation accuracy: 0.0
Average validation loss: 3.371644826039024
[epoch: 1, i:    99] avg mini-batch loss: 3.434
[epoch: 1, i:   199] avg mini-batch loss: 3.409
[epoch: 1, i:   299] avg mini-batch loss: 3.459
Validation accuracy: 0.0
Average validation loss: 3.3670399603636367
[epoch: 2, i:    99] avg mini-batch loss: 3.412
[epoch: 2, i:   199] avg mini-batch loss: 3.395
[epoch: 2, i:   299] avg mini-batch loss: 3.415
Validation accuracy: 0.0
Average validation loss: 3.318961210872816
[epoch: 3, i:    99] avg mini-batch loss: 3.402
[epoch: 3, i:   199] avg mini-batch loss: 3.397
[epoch: 3, i:   299] avg mini-batch loss: 3.372
Validation accuracy: 0.0
Average validation loss: 3.3761776815290037
[epoch: 4, i:    99] avg mini-batch loss: 3.387
[epoch: 4, i:   199] avg mini-batch loss: 3.400
[epoch: 4, i:   299] avg mini-batch loss: 3.386
Valida

In [81]:
# vgg16.eval(val)

In [ ]:
plt.plot(avg_losses)
plt.plot(avg_val_losses)

In [85]:
pwd

'/home/jdlevy/xai_birds/notebooks'

In [ ]:
'../'

In [ ]:
torch.save(vgg16.state_dict(), '../models/transfer_vgg16_31_class_500_epoch_4_batch.pth')

In [10]:
torch.load()

<module 'torch' from '/home/jdlevy/anaconda3/envs/py38/lib/python3.8/site-packages/torch/__init__.py'>

In [28]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

**Captum Model Explainability**

In [ ]:
bd.

In [ ]:
inv_classes = {v: k for k, v in }

In [11]:


# The function to show an image.
def imshow(img):
#     img = img / 2 + 0.5     # Unnormalize.
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()



In [ ]:
val_bird_dataset

In [88]:
# Check several images.
dataiter = iter(val_loader)
batch_size = 4
# for i in range(len(dataiter)):
images, labels = dataiter.next()
imshow(torchvision.utils.make_grid(images))
# print(labels)
# print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))
outputs = net(images.to(device))
_, predicted = torch.max(outputs, 1)

print("Ground Truth:", [classes[labels[i]] for i in range(batch_size)])
print("Predicted:",[classes[predicted[i]] for i in range(batch_size)])
print("Accuracy:",sum(np.array(labels)==np.array(predicted.cpu()))/len(labels))

NameError: name 'val_loader' is not defined

In [29]:
from captum.attr import GuidedGradCam

In [30]:
last_conv = vgg16.features[40]

In [31]:
gc = GuidedGradCam(vgg16, last_conv)

In [35]:
gc.attribute?

In [36]:
img, label = val_bird_dataset[0]['image'], val_bird_dataset[0]['label']

In [37]:
gc.attribute(img, label)

/home/jdlevy/anaconda3/envs/py38/lib/python3.8/site-packages/captum/_utils/gradient.py:53: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 3, 3, 3], but got 3-dimensional input of size [3, 224, 224] instead